In [ ]:
#Mount the Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
####Env Setting
#You may run this only once
"""
%cd /content/drive/MyDrive/calvinoffon
!git clone https://github.com/mees/calvin_env.git
%cd /content/drive/MyDrive/calvinoffon/calvin_env
!git clone https://github.com/lukashermann/tacto.git
"""

In [ ]:
# Install packages
%cd /content/drive/MyDrive/calvinoffon/calvin_env/tacto/
!pip3 install -e .
%cd /content/drive/MyDrive/calvinoffon/calvin_env
!pip3 install -e .
!pip3 install -U numpy

In [ ]:
# Run this to check if the installation was succesful
%cd /content/drive/MyDrive/calvinoffon/calvin_env
from calvin_env.envs.play_table_env import PlayTableSimEnv
# 안되면 런타임 세션 다시시작 후, 이 셀부터 다시 실행

In [ ]:
##데이터셋 /calvinoffon/calvin_env/dataset/에 저장
## dataset 폴더 없으면 !mkdir /content/drive/MyDrive/calvinoffon/calvin_env/dataset
##한번만 실행

%cd /content/drive/MyDrive/calvinoffon/
from dataset.calvin_rezip import dataset_rezip
dataset_rezip(data_path="/content/drive/MyDrive/calvinoffon/calvin_env/dataset/calvin_debug_dataset/training",
             save_path='./data',
             file_name='training')

In [ ]:
##데이터셋 잘 로드 되는지 확인
"%cd /content/drive/MyDrive/calvinoffon"
from dataset.calvin_dataset import CALVIN_dataset
training_dataset = CALVIN_dataset('./data/training.npz')
!python dataset/calvin_dataset.py
from torch.utils.data import DataLoader
train_data_loader = DataLoader(dataset=training_dataset, batch_size=2,shuffle=True)

REWARD
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0, 10],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0, 10]])
TASK ID
tensor([30, 16])


In [ ]:
##Offline Setting
%cd /content/drive/MyDrive/calvinoffon
!python offline_train_script.py --cuda

/content/drive/MyDrive/calvinoffon
2024-05-17 07:51:46.310014: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 07:51:46.310064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 07:51:46.311332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 07:51:46.317972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 07:51:4

In [62]:
##Offline Setting
%cd /content/drive/MyDrive/calvinoffon
!python online_train_script.py --cuda

/content/drive/MyDrive/calvinoffon
2024-05-17 09:09:18.018566: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 09:09:18.018615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 09:09:18.019902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 09:09:19.079631: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/drive/MyDrive/calvinoffon/online_train_script.py:69: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for versi

In [58]:
# 저장된 SAC 불러오는 것 구현해야함 지금 구현된건 Random Policy
from easydict import EasyDict
args = EasyDict()
args.gamma = 0
args.tau = 0
args.lr = 0
args.alpha = 0
args.automatic_entropy_tuning = False
args.target_update_interval = 1
args.seed = 123456
args.cuda = True
args.hidden_size =256
args.policy = 'Gaussian'

import time
import hydra
import numpy as np
import os
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from IPython import display

import sys
sys.path.append("./language_soft_actor_critic")
sys.path.append("/content/drive/MyDrive/calvinoffon/calvin_env")

##Evaluation Code
eval_dataset = np.load('./data/training.npz',allow_pickle=True)['arr_0']

##Retrieve agent : agent=torch.load(')

##Environment init
import hydra
from hydra import initialize, compose

with initialize(config_path="./calvin_env/conf/"):
  cfg = compose(config_name="config_data_collection.yaml", overrides=["cameras=static_and_gripper"])
  cfg.env["use_egl"] = False
  cfg.env["show_gui"] = False
  cfg.env["use_vr"] = False
  cfg.env["use_scene_info"] = True
  cfg.env['control_freq'] = 60
  print(cfg.env)

import custom_calvin_env as calvin

new_env_cfg = {**cfg.env}
new_env_cfg["tasks"] = cfg.tasks
new_env_cfg.pop('_target_', None)
new_env_cfg.pop('_recursive_', None)
env = calvin.Custom_Calvin_Env(sparse_reward_val=10,**new_env_cfg)

env.reset()

from language_soft_actor_critic import off2on_sac
####argument pass in Colab

agent =off2on_sac.Off2On_SAC(39,384,32,64,env.action_space,args)

for epi_idx in np.random.permutation(len(eval_dataset)):
  episode = eval_dataset[epi_idx]

  language_embedding = episode['emb']
  init_robot_obs = episode['robot_obs'][0,:]
  init_scene_obs = episode['scene_obs'][0,:]
  task = episode['task']

  observation = env.reset(scene_obs=init_scene_obs,robot_obs=init_robot_obs)
  max_epi_length = 64

  curr_epi_length = 0
  done = False
  while not done and curr_epi_length < max_epi_length:
    #print(observation['robot_obs'].shape)
    #print(observation['scene_obs'].shape)
    #print(language_embedding.shape)
    #print(np.concatenate([observation['robot_obs'],observation['scene_obs']]).shape)
    state = np.concatenate([observation['robot_obs'],observation['scene_obs']]).reshape(1,-1)

    action = agent.select_action(state,language_embedding)
    if(action[-1]>0):
      action[-1]=1
    else:
      action[-1]=-1

    print(action)

    observation, reward, done, info = env.step(action)
    #The observation is given as a dictionary with different values
    rgb = env.render(mode="rgb_array")[:,:,:]#::-1
    display.clear_output(wait=True)
    display.display(plt.gcf())
    print(f"Current Task - {task}")
    plt.imshow(rgb)
    curr_epi_length += 1

<ipython-input-58-3609cd28ed26>:36: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="./calvin_env/conf/"):
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_data_collection.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/hydra/core/default_element.py:124: UserWarning: In 'vr_input/vr_controller/vive': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(


{'_target_': 'calvin_env.envs.play_table_env.PlayTableSimEnv', '_recursive_': False, 'cameras': '${cameras}', 'seed': 0, 'bullet_time_step': 240.0, 'use_vr': False, 'show_gui': False, 'robot_cfg': '${robot}', 'scene_cfg': '${scene}', 'use_scene_info': True, 'use_egl': False, 'control_freq': 60}
WARNING uncommitted modified files: calvin_env/scripts/convert_gripper_actions.py,calvin_env/scripts/render_low_freq.py,data/franka_panda/meshes/visual/~$Assem1.SLDASM,data/franka_panda/meshes/visual/~$FRANKA_Finger.SLDPRT,data/franka_panda/meshes/visual/~$finger.SLDPRT,egl_check/build.sh,tacto
disconnecting id 2 from server
[-0.92174053  0.879079   -0.3074301   0.72753614  0.3311897   0.40107214
  1.        ]


KeyboardInterrupt: 